In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
from skimage.exposure import equalize_adapthist
from skimage.restoration import denoise_tv_chambolle

def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2
        
        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())
        
        img = np.dstack([r,g,b])
        img = denoise_tv_chambolle(img,weight=0.2,multichannel=True)
        #img = std_img(img)
        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.0008
    elif epoch<30:
        return 0.0004
    else:
        return 0.0001
    

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=150, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.64963, saving model to best_m.h5
 - 3s - loss: 0.6925 - acc: 0.5330 - val_loss: 0.6496 - val_acc: 0.7009
Epoch 2/150
Epoch 00002: val_loss did not improve
 - 1s - loss: 0.6479 - acc: 0.6294 - val_loss: 0.6579 - val_acc: 0.5140
Epoch 3/150
Epoch 00003: val_loss improved from 0.64963 to 0.60325, saving model to best_m.h5
 - 1s - loss: 0.6054 - acc: 0.6222 - val_loss: 0.6033 - val_acc: 0.6710
Epoch 4/150
Epoch 00004: val_loss did not improve
 - 1s - loss: 0.5960 - acc: 0.6484 - val_loss: 0.6734 - val_acc: 0.6280
Epoch 5/150
Epoch 00005: val_loss improved from 0.60325 to 0.58333, saving model to best_m.h5
 - 1s - loss: 0.6018 - acc: 0.6515 - val_loss: 0.5833 - val_acc: 0.6841
Epoch 6/150
Epoch 00006: val_loss improved from 0.58333 to 0.58120, saving model to best_m.h5
 - 1s - loss: 0.5820 - acc: 0.6834 - val_loss: 0.5812 - val_acc: 0.6860
Epoch 7/150
Epoch 00007: val_loss improved from 0.58120 to 0.56893, saving model to best_m.h5
 -

Epoch 00058: val_loss improved from 0.30836 to 0.30592, saving model to best_m.h5
 - 1s - loss: 0.3542 - acc: 0.8343 - val_loss: 0.3059 - val_acc: 0.8449
Epoch 59/150
Epoch 00059: val_loss did not improve
 - 1s - loss: 0.3514 - acc: 0.8287 - val_loss: 0.3072 - val_acc: 0.8393
Epoch 60/150
Epoch 00060: val_loss did not improve
 - 1s - loss: 0.3409 - acc: 0.8239 - val_loss: 0.3062 - val_acc: 0.8449
Epoch 61/150
Epoch 00061: val_loss improved from 0.30592 to 0.30437, saving model to best_m.h5
 - 1s - loss: 0.3422 - acc: 0.8289 - val_loss: 0.3044 - val_acc: 0.8411
Epoch 62/150
Epoch 00062: val_loss improved from 0.30437 to 0.30243, saving model to best_m.h5
 - 1s - loss: 0.3465 - acc: 0.8289 - val_loss: 0.3024 - val_acc: 0.8393
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.3397 - acc: 0.8376 - val_loss: 0.3124 - val_acc: 0.8449
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.3550 - acc: 0.8327 - val_loss: 0.3033 - val_acc: 0.8467
Epoch 65/150
Epoch 0

Epoch 121/150
Epoch 00121: val_loss did not improve
 - 1s - loss: 0.3066 - acc: 0.8426 - val_loss: 0.2827 - val_acc: 0.8561
Epoch 122/150
Epoch 00122: val_loss did not improve
 - 1s - loss: 0.3106 - acc: 0.8518 - val_loss: 0.2883 - val_acc: 0.8430
Epoch 123/150
Epoch 00123: val_loss did not improve
 - 1s - loss: 0.3244 - acc: 0.8490 - val_loss: 0.2969 - val_acc: 0.8467
Epoch 124/150
Epoch 00124: val_loss did not improve
 - 1s - loss: 0.3215 - acc: 0.8355 - val_loss: 0.2772 - val_acc: 0.8636
Epoch 125/150
Epoch 00125: val_loss did not improve
 - 1s - loss: 0.3205 - acc: 0.8426 - val_loss: 0.2769 - val_acc: 0.8542
Epoch 126/150
Epoch 00126: val_loss did not improve
 - 1s - loss: 0.3046 - acc: 0.8561 - val_loss: 0.2833 - val_acc: 0.8505
Epoch 127/150
Epoch 00127: val_loss improved from 0.27669 to 0.27555, saving model to best_m.h5
 - 1s - loss: 0.3340 - acc: 0.8346 - val_loss: 0.2755 - val_acc: 0.8561
Epoch 128/150
Epoch 00128: val_loss did not improve
 - 1s - loss: 0.3300 - acc: 0.8303 -

Epoch 30/150
Epoch 00030: val_loss did not improve
 - 1s - loss: 0.4400 - acc: 0.7761 - val_loss: 0.4009 - val_acc: 0.8112
Epoch 31/150
Epoch 00031: val_loss did not improve
 - 1s - loss: 0.4079 - acc: 0.7885 - val_loss: 0.3746 - val_acc: 0.8000
Epoch 32/150
Epoch 00032: val_loss did not improve
 - 1s - loss: 0.4310 - acc: 0.7813 - val_loss: 0.3793 - val_acc: 0.8131
Epoch 33/150
Epoch 00033: val_loss did not improve
 - 1s - loss: 0.3995 - acc: 0.8128 - val_loss: 0.3750 - val_acc: 0.8112
Epoch 34/150
Epoch 00034: val_loss improved from 0.35382 to 0.34920, saving model to best_m.h5
 - 1s - loss: 0.4233 - acc: 0.7888 - val_loss: 0.3492 - val_acc: 0.8299
Epoch 35/150
Epoch 00035: val_loss did not improve
 - 1s - loss: 0.4183 - acc: 0.7938 - val_loss: 0.3605 - val_acc: 0.8243
Epoch 36/150
Epoch 00036: val_loss did not improve
 - 1s - loss: 0.3851 - acc: 0.7945 - val_loss: 0.3493 - val_acc: 0.8393
Epoch 37/150
Epoch 00037: val_loss did not improve
 - 1s - loss: 0.3984 - acc: 0.7984 - val_los

Epoch 00094: val_loss did not improve
 - 1s - loss: 0.3733 - acc: 0.8187 - val_loss: 0.3395 - val_acc: 0.8355
Epoch 95/150
Epoch 00095: val_loss did not improve
 - 1s - loss: 0.3596 - acc: 0.8384 - val_loss: 0.3512 - val_acc: 0.8355
Epoch 96/150
Epoch 00096: val_loss did not improve
 - 1s - loss: 0.3429 - acc: 0.8443 - val_loss: 0.3335 - val_acc: 0.8505
Epoch 97/150
Epoch 00097: val_loss did not improve
 - 1s - loss: 0.3822 - acc: 0.7986 - val_loss: 0.3551 - val_acc: 0.8355
Epoch 98/150
Epoch 00098: val_loss did not improve
 - 1s - loss: 0.3579 - acc: 0.8291 - val_loss: 0.3431 - val_acc: 0.8280
Epoch 99/150
Epoch 00099: val_loss did not improve
 - 1s - loss: 0.3661 - acc: 0.8206 - val_loss: 0.3322 - val_acc: 0.8299
Epoch 100/150
Epoch 00100: val_loss did not improve
 - 1s - loss: 0.3970 - acc: 0.8134 - val_loss: 0.3426 - val_acc: 0.8374
Epoch 101/150
Epoch 00101: val_loss did not improve
 - 1s - loss: 0.3666 - acc: 0.8192 - val_loss: 0.3390 - val_acc: 0.8374
Epoch 102/150
Epoch 00102: 

Epoch 00008: val_loss improved from 0.60712 to 0.60200, saving model to best_m.h5
 - 1s - loss: 0.5791 - acc: 0.6837 - val_loss: 0.6020 - val_acc: 0.6648
Epoch 9/150
Epoch 00009: val_loss improved from 0.60200 to 0.56069, saving model to best_m.h5
 - 1s - loss: 0.5703 - acc: 0.6801 - val_loss: 0.5607 - val_acc: 0.6779
Epoch 10/150
Epoch 00010: val_loss improved from 0.56069 to 0.47787, saving model to best_m.h5
 - 1s - loss: 0.5176 - acc: 0.7296 - val_loss: 0.4779 - val_acc: 0.7622
Epoch 11/150
Epoch 00011: val_loss did not improve
 - 1s - loss: 0.5077 - acc: 0.7257 - val_loss: 0.4894 - val_acc: 0.7285
Epoch 12/150
Epoch 00012: val_loss improved from 0.47787 to 0.45333, saving model to best_m.h5
 - 1s - loss: 0.4680 - acc: 0.7589 - val_loss: 0.4533 - val_acc: 0.7566
Epoch 13/150
Epoch 00013: val_loss did not improve
 - 1s - loss: 0.4811 - acc: 0.7439 - val_loss: 0.5140 - val_acc: 0.7079
Epoch 14/150
Epoch 00014: val_loss improved from 0.45333 to 0.43869, saving model to best_m.h5
 - 1s

Epoch 68/150
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.3170 - acc: 0.8548 - val_loss: 0.3475 - val_acc: 0.8371
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.3163 - acc: 0.8577 - val_loss: 0.3455 - val_acc: 0.8296
Epoch 70/150
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.3312 - acc: 0.8398 - val_loss: 0.3542 - val_acc: 0.8315
Epoch 71/150
Epoch 00071: val_loss improved from 0.33790 to 0.33357, saving model to best_m.h5
 - 1s - loss: 0.3062 - acc: 0.8567 - val_loss: 0.3336 - val_acc: 0.8427
Epoch 72/150
Epoch 00072: val_loss did not improve
 - 1s - loss: 0.3131 - acc: 0.8669 - val_loss: 0.3475 - val_acc: 0.8446
Epoch 73/150
Epoch 00073: val_loss did not improve
 - 1s - loss: 0.3257 - acc: 0.8383 - val_loss: 0.3458 - val_acc: 0.8333
Epoch 74/150
Epoch 00074: val_loss did not improve
 - 1s - loss: 0.3251 - acc: 0.8444 - val_loss: 0.3353 - val_acc: 0.8296
Epoch 75/150
Epoch 00075: val_loss did not improve
 - 1s - loss: 0.3176 - acc: 0.8482 - val_los

Epoch 132/150
Epoch 00132: val_loss improved from 0.32445 to 0.31913, saving model to best_m.h5
 - 1s - loss: 0.2703 - acc: 0.8736 - val_loss: 0.3191 - val_acc: 0.8464
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 1s - loss: 0.2744 - acc: 0.8768 - val_loss: 0.3240 - val_acc: 0.8521
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 1s - loss: 0.2920 - acc: 0.8736 - val_loss: 0.3599 - val_acc: 0.8390
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 1s - loss: 0.2766 - acc: 0.8768 - val_loss: 0.3333 - val_acc: 0.8371
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 1s - loss: 0.2804 - acc: 0.8650 - val_loss: 0.3511 - val_acc: 0.8333
Epoch 137/150
Epoch 00137: val_loss did not improve
 - 1s - loss: 0.2944 - acc: 0.8736 - val_loss: 0.3257 - val_acc: 0.8577
Epoch 138/150
Epoch 00138: val_loss did not improve
 - 1s - loss: 0.2834 - acc: 0.8719 - val_loss: 0.3266 - val_acc: 0.8521
Epoch 139/150
Epoch 00139: val_loss did not improve
 - 1s - loss: 0.2882 - acc: 0.8692 -

In [6]:
import pickle
with open('../features/cnn_1_aug_skimage_denoise_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))

#pre 3219
# new 2965

# skimage 2725

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_1_aug_denoise_preprocess.csv', index=False)

0.303949278236
         id  is_iceberg
0  5941774d    0.407001
1  4023181e    0.537552
2  b20200e4    0.653724
3  e7f018bb    0.967307
4  4371c8c3    0.496501


In [7]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')

model model


In [8]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=150, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.69584, saving model to best_m.h5
 - 2s - loss: 0.6927 - acc: 0.5197 - val_loss: 0.6958 - val_acc: 0.5140
Epoch 2/150
Epoch 00002: val_loss improved from 0.69584 to 0.61514, saving model to best_m.h5
 - 1s - loss: 0.6721 - acc: 0.5484 - val_loss: 0.6151 - val_acc: 0.6056
Epoch 3/150
Epoch 00003: val_loss improved from 0.61514 to 0.54141, saving model to best_m.h5
 - 1s - loss: 0.5839 - acc: 0.6733 - val_loss: 0.5414 - val_acc: 0.7159
Epoch 4/150
Epoch 00004: val_loss improved from 0.54141 to 0.52875, saving model to best_m.h5
 - 1s - loss: 0.5405 - acc: 0.7120 - val_loss: 0.5287 - val_acc: 0.7346
Epoch 5/150
Epoch 00005: val_loss improved from 0.52875 to 0.50271, saving model to best_m.h5
 - 1s - loss: 0.5026 - acc: 0.7365 - val_loss: 0.5027 - val_acc: 0.7551
Epoch 6/150
Epoch 00006: val_loss improved from 0.50271 to 0.49677, saving model to best_m.h5
 - 1s - loss: 0.5081 - acc: 0.7536 - val_loss: 0.4968 - val_acc: 0.7664
Epoch 7/

Epoch 60/150
Epoch 00060: val_loss improved from 0.26254 to 0.26054, saving model to best_m.h5
 - 1s - loss: 0.2649 - acc: 0.8719 - val_loss: 0.2605 - val_acc: 0.8692
Epoch 61/150
Epoch 00061: val_loss did not improve
 - 1s - loss: 0.3008 - acc: 0.8549 - val_loss: 0.3032 - val_acc: 0.8636
Epoch 62/150
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.2584 - acc: 0.8842 - val_loss: 0.2785 - val_acc: 0.8673
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.2787 - acc: 0.8611 - val_loss: 0.2781 - val_acc: 0.8654
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.2663 - acc: 0.8852 - val_loss: 0.2851 - val_acc: 0.8692
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.2809 - acc: 0.8717 - val_loss: 0.2656 - val_acc: 0.8804
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.2629 - acc: 0.8802 - val_loss: 0.2752 - val_acc: 0.8692
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.2675 - acc: 0.8692 - val_los

Epoch 124/150
Epoch 00124: val_loss did not improve
 - 1s - loss: 0.2380 - acc: 0.8909 - val_loss: 0.2502 - val_acc: 0.8766
Epoch 125/150
Epoch 00125: val_loss did not improve
 - 1s - loss: 0.2461 - acc: 0.8904 - val_loss: 0.2560 - val_acc: 0.8860
Epoch 126/150
Epoch 00126: val_loss did not improve
 - 1s - loss: 0.2394 - acc: 0.8939 - val_loss: 0.2721 - val_acc: 0.8673
Epoch 127/150
Epoch 00127: val_loss did not improve
 - 1s - loss: 0.2477 - acc: 0.8899 - val_loss: 0.2543 - val_acc: 0.8785
Epoch 128/150
Epoch 00128: val_loss did not improve
 - 1s - loss: 0.2458 - acc: 0.8883 - val_loss: 0.2530 - val_acc: 0.8879
Epoch 129/150
Epoch 00129: val_loss did not improve
 - 1s - loss: 0.2472 - acc: 0.8759 - val_loss: 0.2568 - val_acc: 0.8822
Epoch 130/150
Epoch 00130: val_loss did not improve
 - 1s - loss: 0.2733 - acc: 0.8672 - val_loss: 0.2685 - val_acc: 0.8879
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 1s - loss: 0.2476 - acc: 0.8864 - val_loss: 0.2533 - val_acc: 0.8897
Epoch 13

Epoch 34/150
Epoch 00034: val_loss did not improve
 - 1s - loss: 0.3512 - acc: 0.8263 - val_loss: 0.3499 - val_acc: 0.8486
Epoch 35/150
Epoch 00035: val_loss did not improve
 - 1s - loss: 0.3404 - acc: 0.8494 - val_loss: 0.3494 - val_acc: 0.8449
Epoch 36/150
Epoch 00036: val_loss improved from 0.34851 to 0.34582, saving model to best_m.h5
 - 1s - loss: 0.3620 - acc: 0.8220 - val_loss: 0.3458 - val_acc: 0.8486
Epoch 37/150
Epoch 00037: val_loss did not improve
 - 1s - loss: 0.3384 - acc: 0.8535 - val_loss: 0.3521 - val_acc: 0.8374
Epoch 38/150
Epoch 00038: val_loss improved from 0.34582 to 0.34560, saving model to best_m.h5
 - 1s - loss: 0.3404 - acc: 0.8407 - val_loss: 0.3456 - val_acc: 0.8411
Epoch 39/150
Epoch 00039: val_loss did not improve
 - 1s - loss: 0.3623 - acc: 0.8289 - val_loss: 0.3493 - val_acc: 0.8523
Epoch 40/150
Epoch 00040: val_loss improved from 0.34560 to 0.34018, saving model to best_m.h5
 - 1s - loss: 0.3441 - acc: 0.8362 - val_loss: 0.3402 - val_acc: 0.8430
Epoch 4

Epoch 94/150
Epoch 00094: val_loss did not improve
 - 1s - loss: 0.2962 - acc: 0.8516 - val_loss: 0.2969 - val_acc: 0.8710
Epoch 95/150
Epoch 00095: val_loss did not improve
 - 1s - loss: 0.3155 - acc: 0.8568 - val_loss: 0.3082 - val_acc: 0.8579
Epoch 96/150
Epoch 00096: val_loss did not improve
 - 1s - loss: 0.3068 - acc: 0.8456 - val_loss: 0.3242 - val_acc: 0.8617
Epoch 97/150
Epoch 00097: val_loss did not improve
 - 1s - loss: 0.3010 - acc: 0.8644 - val_loss: 0.2994 - val_acc: 0.8692
Epoch 98/150
Epoch 00098: val_loss did not improve
 - 1s - loss: 0.2841 - acc: 0.8693 - val_loss: 0.2948 - val_acc: 0.8748
Epoch 99/150
Epoch 00099: val_loss did not improve
 - 1s - loss: 0.3065 - acc: 0.8511 - val_loss: 0.2997 - val_acc: 0.8729
Epoch 100/150
Epoch 00100: val_loss did not improve
 - 1s - loss: 0.3090 - acc: 0.8606 - val_loss: 0.3007 - val_acc: 0.8748
Epoch 101/150
Epoch 00101: val_loss did not improve
 - 1s - loss: 0.3023 - acc: 0.8690 - val_loss: 0.2937 - val_acc: 0.8692
Epoch 102/150


Epoch 00006: val_loss improved from 0.52649 to 0.50186, saving model to best_m.h5
 - 1s - loss: 0.5395 - acc: 0.7259 - val_loss: 0.5019 - val_acc: 0.7697
Epoch 7/150
Epoch 00007: val_loss improved from 0.50186 to 0.45300, saving model to best_m.h5
 - 1s - loss: 0.4827 - acc: 0.7592 - val_loss: 0.4530 - val_acc: 0.7884
Epoch 8/150
Epoch 00008: val_loss did not improve
 - 1s - loss: 0.5050 - acc: 0.7381 - val_loss: 0.4616 - val_acc: 0.7753
Epoch 9/150
Epoch 00009: val_loss improved from 0.45300 to 0.40572, saving model to best_m.h5
 - 1s - loss: 0.4356 - acc: 0.7907 - val_loss: 0.4057 - val_acc: 0.8034
Epoch 10/150
Epoch 00010: val_loss did not improve
 - 1s - loss: 0.4442 - acc: 0.7894 - val_loss: 0.4660 - val_acc: 0.7790
Epoch 11/150
Epoch 00011: val_loss improved from 0.40572 to 0.39322, saving model to best_m.h5
 - 1s - loss: 0.4135 - acc: 0.8055 - val_loss: 0.3932 - val_acc: 0.8146
Epoch 12/150
Epoch 00012: val_loss improved from 0.39322 to 0.38977, saving model to best_m.h5
 - 1s -

Epoch 66/150
Epoch 00066: val_loss improved from 0.29578 to 0.29567, saving model to best_m.h5
 - 1s - loss: 0.2447 - acc: 0.8945 - val_loss: 0.2957 - val_acc: 0.8708
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.2519 - acc: 0.8833 - val_loss: 0.2991 - val_acc: 0.8689
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.2516 - acc: 0.8927 - val_loss: 0.3081 - val_acc: 0.8614
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.2672 - acc: 0.8750 - val_loss: 0.2984 - val_acc: 0.8727
Epoch 70/150
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.2409 - acc: 0.8939 - val_loss: 0.3001 - val_acc: 0.8764
Epoch 71/150
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.2491 - acc: 0.8918 - val_loss: 0.2986 - val_acc: 0.8689
Epoch 72/150
Epoch 00072: val_loss did not improve
 - 1s - loss: 0.2569 - acc: 0.8788 - val_loss: 0.3021 - val_acc: 0.8708
Epoch 73/150
Epoch 00073: val_loss did not improve
 - 1s - loss: 0.2727 - acc: 0.8686 - val_los

Epoch 132/150
Epoch 00132: val_loss did not improve
 - 1s - loss: 0.2352 - acc: 0.8892 - val_loss: 0.3204 - val_acc: 0.8689
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 1s - loss: 0.2294 - acc: 0.8893 - val_loss: 0.3138 - val_acc: 0.8539
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 1s - loss: 0.2267 - acc: 0.8919 - val_loss: 0.3108 - val_acc: 0.8727
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 1s - loss: 0.2338 - acc: 0.8958 - val_loss: 0.3112 - val_acc: 0.8502
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 1s - loss: 0.2263 - acc: 0.8868 - val_loss: 0.3098 - val_acc: 0.8689
Epoch 137/150
Epoch 00137: val_loss did not improve
 - 1s - loss: 0.2190 - acc: 0.8984 - val_loss: 0.3352 - val_acc: 0.8596
Epoch 138/150
Epoch 00138: val_loss did not improve
 - 1s - loss: 0.2265 - acc: 0.8926 - val_loss: 0.3102 - val_acc: 0.8633
Epoch 139/150
Epoch 00139: val_loss did not improve
 - 1s - loss: 0.2101 - acc: 0.9098 - val_loss: 0.3228 - val_acc: 0.8652
Epoch 14

In [9]:
with open('../features/cnn_2_aug_denoise_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2728
# skimage 2710

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_2_aug_denoise_preprocess.csv', index=False)

0.270209624354
         id  is_iceberg
0  5941774d    0.728009
1  4023181e    0.379872
2  b20200e4    0.918563
3  e7f018bb    0.990525
4  4371c8c3    0.226310


In [10]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')

model model


In [11]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=150, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.69227, saving model to best_m.h5
 - 3s - loss: 0.6946 - acc: 0.5280 - val_loss: 0.6923 - val_acc: 0.5140
Epoch 2/150
Epoch 00002: val_loss improved from 0.69227 to 0.67507, saving model to best_m.h5
 - 2s - loss: 0.6806 - acc: 0.5785 - val_loss: 0.6751 - val_acc: 0.5140
Epoch 3/150
Epoch 00003: val_loss improved from 0.67507 to 0.61935, saving model to best_m.h5
 - 2s - loss: 0.6485 - acc: 0.5863 - val_loss: 0.6194 - val_acc: 0.5776
Epoch 4/150
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.6106 - acc: 0.6464 - val_loss: 0.6533 - val_acc: 0.6654
Epoch 5/150
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.5980 - acc: 0.6758 - val_loss: 0.6218 - val_acc: 0.6636
Epoch 6/150
Epoch 00006: val_loss improved from 0.61935 to 0.58477, saving model to best_m.h5
 - 2s - loss: 0.5950 - acc: 0.6726 - val_loss: 0.5848 - val_acc: 0.7028
Epoch 7/150
Epoch 00007: val_loss improved from 0.58477 to 0.55321, saving model to best_m.h5
 -

Epoch 59/150
Epoch 00059: val_loss did not improve
 - 2s - loss: 0.3276 - acc: 0.8473 - val_loss: 0.3306 - val_acc: 0.8374
Epoch 60/150
Epoch 00060: val_loss improved from 0.32062 to 0.31847, saving model to best_m.h5
 - 2s - loss: 0.3534 - acc: 0.8384 - val_loss: 0.3185 - val_acc: 0.8598
Epoch 61/150
Epoch 00061: val_loss did not improve
 - 2s - loss: 0.3275 - acc: 0.8478 - val_loss: 0.3396 - val_acc: 0.8262
Epoch 62/150
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.3375 - acc: 0.8376 - val_loss: 0.3187 - val_acc: 0.8598
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.3383 - acc: 0.8206 - val_loss: 0.3358 - val_acc: 0.8430
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.3505 - acc: 0.8315 - val_loss: 0.3207 - val_acc: 0.8449
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.3270 - acc: 0.8376 - val_loss: 0.3360 - val_acc: 0.8486
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.3208 - acc: 0.8395 - val_los

Epoch 121/150
Epoch 00121: val_loss did not improve
 - 2s - loss: 0.3014 - acc: 0.8587 - val_loss: 0.3237 - val_acc: 0.8561
Epoch 122/150
Epoch 00122: val_loss did not improve
 - 2s - loss: 0.2874 - acc: 0.8717 - val_loss: 0.2938 - val_acc: 0.8692
Epoch 123/150
Epoch 00123: val_loss did not improve
 - 2s - loss: 0.3057 - acc: 0.8582 - val_loss: 0.3288 - val_acc: 0.8636
Epoch 124/150
Epoch 00124: val_loss did not improve
 - 2s - loss: 0.3033 - acc: 0.8592 - val_loss: 0.3039 - val_acc: 0.8636
Epoch 125/150
Epoch 00125: val_loss did not improve
 - 2s - loss: 0.2758 - acc: 0.8596 - val_loss: 0.3127 - val_acc: 0.8579
Epoch 126/150
Epoch 00126: val_loss did not improve
 - 2s - loss: 0.3118 - acc: 0.8403 - val_loss: 0.3172 - val_acc: 0.8486
Epoch 127/150
Epoch 00127: val_loss improved from 0.29344 to 0.29125, saving model to best_m.h5
 - 2s - loss: 0.2961 - acc: 0.8672 - val_loss: 0.2912 - val_acc: 0.8710
Epoch 128/150
Epoch 00128: val_loss did not improve
 - 2s - loss: 0.2887 - acc: 0.8554 -

Epoch 32/150
Epoch 00032: val_loss improved from 0.30861 to 0.29790, saving model to best_m.h5
 - 2s - loss: 0.3328 - acc: 0.8370 - val_loss: 0.2979 - val_acc: 0.8449
Epoch 33/150
Epoch 00033: val_loss improved from 0.29790 to 0.29624, saving model to best_m.h5
 - 2s - loss: 0.3448 - acc: 0.8277 - val_loss: 0.2962 - val_acc: 0.8486
Epoch 34/150
Epoch 00034: val_loss did not improve
 - 2s - loss: 0.3253 - acc: 0.8471 - val_loss: 0.2972 - val_acc: 0.8542
Epoch 35/150
Epoch 00035: val_loss did not improve
 - 2s - loss: 0.3873 - acc: 0.8285 - val_loss: 0.3082 - val_acc: 0.8411
Epoch 36/150
Epoch 00036: val_loss did not improve
 - 2s - loss: 0.3666 - acc: 0.8164 - val_loss: 0.3037 - val_acc: 0.8467
Epoch 37/150
Epoch 00037: val_loss did not improve
 - 2s - loss: 0.3491 - acc: 0.8348 - val_loss: 0.3016 - val_acc: 0.8505
Epoch 38/150
Epoch 00038: val_loss did not improve
 - 2s - loss: 0.3765 - acc: 0.8172 - val_loss: 0.3289 - val_acc: 0.8430
Epoch 39/150
Epoch 00039: val_loss improved from 0.

Epoch 95/150
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.3305 - acc: 0.8248 - val_loss: 0.2844 - val_acc: 0.8692
Epoch 96/150
Epoch 00096: val_loss did not improve
 - 2s - loss: 0.2693 - acc: 0.8793 - val_loss: 0.2799 - val_acc: 0.8598
Epoch 97/150
Epoch 00097: val_loss did not improve
 - 2s - loss: 0.3248 - acc: 0.8329 - val_loss: 0.2778 - val_acc: 0.8692
Epoch 98/150
Epoch 00098: val_loss did not improve
 - 2s - loss: 0.3089 - acc: 0.8483 - val_loss: 0.2725 - val_acc: 0.8710
Epoch 99/150
Epoch 00099: val_loss did not improve
 - 2s - loss: 0.2994 - acc: 0.8660 - val_loss: 0.2763 - val_acc: 0.8598
Epoch 100/150
Epoch 00100: val_loss did not improve
 - 2s - loss: 0.3431 - acc: 0.8334 - val_loss: 0.2798 - val_acc: 0.8748
Epoch 101/150
Epoch 00101: val_loss did not improve
 - 2s - loss: 0.3031 - acc: 0.8523 - val_loss: 0.2846 - val_acc: 0.8636
Epoch 102/150
Epoch 00102: val_loss did not improve
 - 2s - loss: 0.2846 - acc: 0.8649 - val_loss: 0.2767 - val_acc: 0.8673
Epoch 103/150

Epoch 8/150
Epoch 00008: val_loss did not improve
 - 2s - loss: 0.6043 - acc: 0.6579 - val_loss: 0.5789 - val_acc: 0.6517
Epoch 9/150
Epoch 00009: val_loss did not improve
 - 2s - loss: 0.5845 - acc: 0.6864 - val_loss: 0.5669 - val_acc: 0.6966
Epoch 10/150
Epoch 00010: val_loss improved from 0.55199 to 0.55012, saving model to best_m.h5
 - 2s - loss: 0.5650 - acc: 0.6893 - val_loss: 0.5501 - val_acc: 0.7360
Epoch 11/150
Epoch 00011: val_loss improved from 0.55012 to 0.53673, saving model to best_m.h5
 - 2s - loss: 0.5323 - acc: 0.7334 - val_loss: 0.5367 - val_acc: 0.7584
Epoch 12/150
Epoch 00012: val_loss improved from 0.53673 to 0.46209, saving model to best_m.h5
 - 2s - loss: 0.5400 - acc: 0.7315 - val_loss: 0.4621 - val_acc: 0.7978
Epoch 13/150
Epoch 00013: val_loss did not improve
 - 2s - loss: 0.5038 - acc: 0.7357 - val_loss: 0.4849 - val_acc: 0.7678
Epoch 14/150
Epoch 00014: val_loss improved from 0.46209 to 0.44211, saving model to best_m.h5
 - 2s - loss: 0.5037 - acc: 0.7620 - 

Epoch 67/150
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.2985 - acc: 0.8589 - val_loss: 0.3225 - val_acc: 0.8633
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.3118 - acc: 0.8529 - val_loss: 0.3230 - val_acc: 0.8633
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.2875 - acc: 0.8616 - val_loss: 0.3210 - val_acc: 0.8577
Epoch 70/150
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.2943 - acc: 0.8557 - val_loss: 0.3230 - val_acc: 0.8596
Epoch 71/150
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.3188 - acc: 0.8474 - val_loss: 0.3222 - val_acc: 0.8614
Epoch 72/150
Epoch 00072: val_loss did not improve
 - 2s - loss: 0.2915 - acc: 0.8640 - val_loss: 0.3444 - val_acc: 0.8483
Epoch 73/150
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.3087 - acc: 0.8566 - val_loss: 0.3222 - val_acc: 0.8708
Epoch 74/150
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.2942 - acc: 0.8623 - val_loss: 0.3240 - val_acc: 0.8596
Epoch 75/150
Epo

Epoch 133/150
Epoch 00133: val_loss did not improve
 - 2s - loss: 0.2826 - acc: 0.8670 - val_loss: 0.3162 - val_acc: 0.8614
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 2s - loss: 0.2610 - acc: 0.8796 - val_loss: 0.3264 - val_acc: 0.8614
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 2s - loss: 0.2563 - acc: 0.8880 - val_loss: 0.3263 - val_acc: 0.8596
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 2s - loss: 0.2760 - acc: 0.8697 - val_loss: 0.3207 - val_acc: 0.8652
Epoch 137/150
Epoch 00137: val_loss did not improve
 - 2s - loss: 0.2754 - acc: 0.8673 - val_loss: 0.3322 - val_acc: 0.8614
Epoch 138/150
Epoch 00138: val_loss did not improve
 - 2s - loss: 0.2658 - acc: 0.8682 - val_loss: 0.3235 - val_acc: 0.8670
Epoch 139/150
Epoch 00139: val_loss did not improve
 - 2s - loss: 0.2764 - acc: 0.8624 - val_loss: 0.3132 - val_acc: 0.8689
Epoch 140/150
Epoch 00140: val_loss did not improve
 - 2s - loss: 0.2669 - acc: 0.8843 - val_loss: 0.3318 - val_acc: 0.8633
Epoch 14

In [12]:
with open('../features/cnn_3_aug_denoise_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2737
# new  2509

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_3_aug_denoise_preprocess.csv', index=False)


0.287572566058
         id  is_iceberg
0  5941774d    0.673160
1  4023181e    0.489832
2  b20200e4    0.795839
3  e7f018bb    0.981191
4  4371c8c3    0.352923


In [13]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [14]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=150, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.71822, saving model to best_m.h5
 - 3s - loss: 0.6819 - acc: 0.7238 - val_loss: 0.7182 - val_acc: 0.4860
Epoch 2/150
Epoch 00002: val_loss did not improve
 - 2s - loss: 0.4758 - acc: 0.7688 - val_loss: 0.8999 - val_acc: 0.4860
Epoch 3/150
Epoch 00003: val_loss improved from 0.71822 to 0.68662, saving model to best_m.h5
 - 2s - loss: 0.4695 - acc: 0.7790 - val_loss: 0.6866 - val_acc: 0.4860
Epoch 4/150
Epoch 00004: val_loss improved from 0.68662 to 0.65800, saving model to best_m.h5
 - 2s - loss: 0.4420 - acc: 0.7823 - val_loss: 0.6580 - val_acc: 0.5477
Epoch 5/150
Epoch 00005: val_loss improved from 0.65800 to 0.58730, saving model to best_m.h5
 - 2s - loss: 0.3962 - acc: 0.8180 - val_loss: 0.5873 - val_acc: 0.6224
Epoch 6/150
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.4211 - acc: 0.8000 - val_loss: 0.6496 - val_acc: 0.5981
Epoch 7/150
Epoch 00007: val_loss improved from 0.58730 to 0.44901, saving model to best_m.h5
 -

Epoch 62/150
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.2005 - acc: 0.9117 - val_loss: 0.2564 - val_acc: 0.8804
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.1856 - acc: 0.9181 - val_loss: 0.2588 - val_acc: 0.8879
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.1917 - acc: 0.9108 - val_loss: 0.2747 - val_acc: 0.8822
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.1815 - acc: 0.9288 - val_loss: 0.2775 - val_acc: 0.8766
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.1744 - acc: 0.9356 - val_loss: 0.2476 - val_acc: 0.8710
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.1814 - acc: 0.9259 - val_loss: 0.2609 - val_acc: 0.8673
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1812 - acc: 0.9304 - val_loss: 0.2892 - val_acc: 0.8766
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.1854 - acc: 0.9278 - val_loss: 0.2519 - val_acc: 0.8729
Epoch 70/150
Epo

Epoch 129/150
Epoch 00129: val_loss did not improve
 - 4s - loss: 0.1129 - acc: 0.9526 - val_loss: 0.2838 - val_acc: 0.8953
Epoch 130/150
Epoch 00130: val_loss did not improve
 - 5s - loss: 0.1208 - acc: 0.9508 - val_loss: 0.3157 - val_acc: 0.8729
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 4s - loss: 0.1178 - acc: 0.9591 - val_loss: 0.2867 - val_acc: 0.8860
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 3s - loss: 0.1211 - acc: 0.9526 - val_loss: 0.2841 - val_acc: 0.9009
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 4s - loss: 0.1107 - acc: 0.9610 - val_loss: 0.3049 - val_acc: 0.8579
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 4s - loss: 0.1234 - acc: 0.9555 - val_loss: 0.2875 - val_acc: 0.8935
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 4s - loss: 0.1372 - acc: 0.9430 - val_loss: 0.3215 - val_acc: 0.8785
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 4s - loss: 0.1222 - acc: 0.9562 - val_loss: 0.2661 - val_acc: 0.8897
Epoch 13

Epoch 40/150
Epoch 00040: val_loss did not improve
 - 4s - loss: 0.2030 - acc: 0.9124 - val_loss: 0.2747 - val_acc: 0.8860
Epoch 41/150
Epoch 00041: val_loss did not improve
 - 4s - loss: 0.2032 - acc: 0.9146 - val_loss: 0.2598 - val_acc: 0.8822
Epoch 42/150
Epoch 00042: val_loss did not improve
 - 6s - loss: 0.1976 - acc: 0.9162 - val_loss: 0.2550 - val_acc: 0.8879
Epoch 43/150
Epoch 00043: val_loss did not improve
 - 5s - loss: 0.2181 - acc: 0.9065 - val_loss: 0.2744 - val_acc: 0.8879
Epoch 44/150
Epoch 00044: val_loss did not improve
 - 4s - loss: 0.2000 - acc: 0.9157 - val_loss: 0.2500 - val_acc: 0.8804
Epoch 45/150
Epoch 00045: val_loss did not improve
 - 4s - loss: 0.2029 - acc: 0.9138 - val_loss: 0.2638 - val_acc: 0.8916
Epoch 46/150
Epoch 00046: val_loss did not improve
 - 6s - loss: 0.1938 - acc: 0.9240 - val_loss: 0.2469 - val_acc: 0.8822
Epoch 47/150
Epoch 00047: val_loss did not improve
 - 5s - loss: 0.2086 - acc: 0.9111 - val_loss: 0.2864 - val_acc: 0.8841
Epoch 48/150
Epo

Epoch 107/150
Epoch 00107: val_loss did not improve
 - 4s - loss: 0.1263 - acc: 0.9508 - val_loss: 0.2802 - val_acc: 0.8935
Epoch 108/150
Epoch 00108: val_loss did not improve
 - 5s - loss: 0.1373 - acc: 0.9422 - val_loss: 0.2892 - val_acc: 0.8860
Epoch 109/150
Epoch 00109: val_loss did not improve
 - 5s - loss: 0.1260 - acc: 0.9564 - val_loss: 0.2827 - val_acc: 0.8879
Epoch 110/150
Epoch 00110: val_loss did not improve
 - 4s - loss: 0.1151 - acc: 0.9530 - val_loss: 0.3085 - val_acc: 0.8785
Epoch 111/150
Epoch 00111: val_loss did not improve
 - 4s - loss: 0.1478 - acc: 0.9470 - val_loss: 0.2973 - val_acc: 0.8879
Epoch 112/150
Epoch 00112: val_loss did not improve
 - 4s - loss: 0.1254 - acc: 0.9468 - val_loss: 0.3041 - val_acc: 0.8841
Epoch 113/150
Epoch 00113: val_loss did not improve
 - 5s - loss: 0.1427 - acc: 0.9347 - val_loss: 0.2929 - val_acc: 0.8860
Epoch 114/150
Epoch 00114: val_loss did not improve
 - 5s - loss: 0.1351 - acc: 0.9395 - val_loss: 0.3167 - val_acc: 0.8636
Epoch 11

Epoch 20/150
Epoch 00020: val_loss did not improve
 - 4s - loss: 0.2853 - acc: 0.8655 - val_loss: 0.3251 - val_acc: 0.8577
Epoch 21/150
Epoch 00021: val_loss did not improve
 - 2s - loss: 0.2755 - acc: 0.8796 - val_loss: 0.3294 - val_acc: 0.8521
Epoch 22/150
Epoch 00022: val_loss did not improve
 - 4s - loss: 0.3114 - acc: 0.8589 - val_loss: 0.3245 - val_acc: 0.8539
Epoch 23/150
Epoch 00023: val_loss did not improve
 - 4s - loss: 0.2715 - acc: 0.8866 - val_loss: 0.3549 - val_acc: 0.8408
Epoch 24/150
Epoch 00024: val_loss did not improve
 - 4s - loss: 0.2572 - acc: 0.8892 - val_loss: 0.3430 - val_acc: 0.8502
Epoch 25/150
Epoch 00025: val_loss did not improve
 - 4s - loss: 0.2428 - acc: 0.8938 - val_loss: 0.4894 - val_acc: 0.7884
Epoch 26/150
Epoch 00026: val_loss did not improve
 - 4s - loss: 0.2775 - acc: 0.8722 - val_loss: 0.3157 - val_acc: 0.8708
Epoch 27/150
Epoch 00027: val_loss did not improve
 - 4s - loss: 0.2880 - acc: 0.8787 - val_loss: 0.3164 - val_acc: 0.8614
Epoch 28/150
Epo

Epoch 85/150
Epoch 00085: val_loss did not improve
 - 5s - loss: 0.1492 - acc: 0.9410 - val_loss: 0.3057 - val_acc: 0.8652
Epoch 86/150
Epoch 00086: val_loss did not improve
 - 5s - loss: 0.1539 - acc: 0.9375 - val_loss: 0.3140 - val_acc: 0.8652
Epoch 87/150
Epoch 00087: val_loss did not improve
 - 4s - loss: 0.1442 - acc: 0.9421 - val_loss: 0.3314 - val_acc: 0.8689
Epoch 88/150
Epoch 00088: val_loss did not improve
 - 4s - loss: 0.1612 - acc: 0.9271 - val_loss: 0.3079 - val_acc: 0.8670
Epoch 89/150
Epoch 00089: val_loss did not improve
 - 5s - loss: 0.1544 - acc: 0.9298 - val_loss: 0.2994 - val_acc: 0.8689
Epoch 90/150
Epoch 00090: val_loss did not improve
 - 4s - loss: 0.1500 - acc: 0.9309 - val_loss: 0.3336 - val_acc: 0.8558
Epoch 91/150
Epoch 00091: val_loss did not improve
 - 5s - loss: 0.1557 - acc: 0.9394 - val_loss: 0.3230 - val_acc: 0.8689
Epoch 92/150
Epoch 00092: val_loss did not improve
 - 5s - loss: 0.1544 - acc: 0.9386 - val_loss: 0.2871 - val_acc: 0.8783
Epoch 93/150
Epo

In [15]:
with open('../features/cnn_4_aug_denoise_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2146

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_4_aug_denoise_preprocess.csv', index=False)

0.255408503154
         id  is_iceberg
0  5941774d    0.314985
1  4023181e    0.868872
2  b20200e4    0.609802
3  e7f018bb    0.971441
4  4371c8c3    0.665169


In [16]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(75, 75, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [17]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=150, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.75258, saving model to best_m.h5
 - 7s - loss: 0.6281 - acc: 0.6856 - val_loss: 0.7526 - val_acc: 0.5140
Epoch 2/150
Epoch 00002: val_loss did not improve
 - 5s - loss: 0.5102 - acc: 0.7391 - val_loss: 0.9262 - val_acc: 0.5140
Epoch 3/150
Epoch 00003: val_loss did not improve
 - 5s - loss: 0.5034 - acc: 0.7463 - val_loss: 0.8082 - val_acc: 0.5140
Epoch 4/150
Epoch 00004: val_loss did not improve
 - 4s - loss: 0.4866 - acc: 0.7536 - val_loss: 0.8792 - val_acc: 0.5140
Epoch 5/150
Epoch 00005: val_loss did not improve
 - 5s - loss: 0.4830 - acc: 0.7489 - val_loss: 1.0213 - val_acc: 0.5140
Epoch 6/150
Epoch 00006: val_loss improved from 0.75258 to 0.73243, saving model to best_m.h5
 - 5s - loss: 0.4610 - acc: 0.7732 - val_loss: 0.7324 - val_acc: 0.5402
Epoch 7/150
Epoch 00007: val_loss did not improve
 - 5s - loss: 0.4426 - acc: 0.7989 - val_loss: 1.1048 - val_acc: 0.4991
Epoch 8/150
Epoch 00008: val_loss improved from 0.73243 to 0.5

Epoch 64/150
Epoch 00064: val_loss improved from 0.27364 to 0.27344, saving model to best_m.h5
 - 5s - loss: 0.2988 - acc: 0.8596 - val_loss: 0.2734 - val_acc: 0.8879
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 5s - loss: 0.3010 - acc: 0.8627 - val_loss: 0.3379 - val_acc: 0.8449
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 5s - loss: 0.3001 - acc: 0.8552 - val_loss: 0.2766 - val_acc: 0.8785
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.3171 - acc: 0.8575 - val_loss: 0.3020 - val_acc: 0.8673
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.3215 - acc: 0.8611 - val_loss: 0.2940 - val_acc: 0.8617
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 5s - loss: 0.3079 - acc: 0.8606 - val_loss: 0.2781 - val_acc: 0.8729
Epoch 70/150
Epoch 00070: val_loss did not improve
 - 5s - loss: 0.3081 - acc: 0.8454 - val_loss: 0.2856 - val_acc: 0.8729
Epoch 71/150
Epoch 00071: val_loss did not improve
 - 4s - loss: 0.2967 - acc: 0.8528 - val_los

Epoch 129/150
Epoch 00129: val_loss did not improve
 - 5s - loss: 0.2889 - acc: 0.8772 - val_loss: 0.2762 - val_acc: 0.8692
Epoch 130/150
Epoch 00130: val_loss did not improve
 - 5s - loss: 0.2702 - acc: 0.8805 - val_loss: 0.3656 - val_acc: 0.8411
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 5s - loss: 0.2687 - acc: 0.8727 - val_loss: 0.3205 - val_acc: 0.8523
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 5s - loss: 0.2626 - acc: 0.8829 - val_loss: 0.2899 - val_acc: 0.8673
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 5s - loss: 0.2839 - acc: 0.8682 - val_loss: 0.3255 - val_acc: 0.8729
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 5s - loss: 0.2788 - acc: 0.8689 - val_loss: 0.2628 - val_acc: 0.8766
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 5s - loss: 0.2685 - acc: 0.8883 - val_loss: 0.2661 - val_acc: 0.8748
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 6s - loss: 0.2711 - acc: 0.8691 - val_loss: 0.3920 - val_acc: 0.8187
Epoch 13

Epoch 00041: val_loss did not improve
 - 5s - loss: 0.3377 - acc: 0.8438 - val_loss: 0.3062 - val_acc: 0.8523
Epoch 42/150
Epoch 00042: val_loss did not improve
 - 5s - loss: 0.2906 - acc: 0.8598 - val_loss: 0.2891 - val_acc: 0.8636
Epoch 43/150
Epoch 00043: val_loss did not improve
 - 5s - loss: 0.3586 - acc: 0.8410 - val_loss: 0.3040 - val_acc: 0.8505
Epoch 44/150
Epoch 00044: val_loss did not improve
 - 6s - loss: 0.3324 - acc: 0.8542 - val_loss: 0.3101 - val_acc: 0.8430
Epoch 45/150
Epoch 00045: val_loss improved from 0.28665 to 0.27871, saving model to best_m.h5
 - 5s - loss: 0.3346 - acc: 0.8469 - val_loss: 0.2787 - val_acc: 0.8766
Epoch 46/150
Epoch 00046: val_loss did not improve
 - 5s - loss: 0.3273 - acc: 0.8499 - val_loss: 0.3221 - val_acc: 0.8486
Epoch 47/150
Epoch 00047: val_loss did not improve
 - 5s - loss: 0.3210 - acc: 0.8507 - val_loss: 0.2820 - val_acc: 0.8710
Epoch 48/150
Epoch 00048: val_loss did not improve
 - 4s - loss: 0.3127 - acc: 0.8535 - val_loss: 0.2962 - v

Epoch 106/150
Epoch 00106: val_loss did not improve
 - 3s - loss: 0.2775 - acc: 0.8831 - val_loss: 0.2921 - val_acc: 0.8673
Epoch 107/150
Epoch 00107: val_loss did not improve
 - 3s - loss: 0.2930 - acc: 0.8653 - val_loss: 0.3163 - val_acc: 0.8449
Epoch 108/150
Epoch 00108: val_loss did not improve
 - 3s - loss: 0.2830 - acc: 0.8668 - val_loss: 0.4030 - val_acc: 0.8224
Epoch 109/150
Epoch 00109: val_loss did not improve
 - 3s - loss: 0.3033 - acc: 0.8618 - val_loss: 0.3292 - val_acc: 0.8449
Epoch 110/150
Epoch 00110: val_loss did not improve
 - 3s - loss: 0.2647 - acc: 0.8909 - val_loss: 0.2946 - val_acc: 0.8654
Epoch 111/150
Epoch 00111: val_loss did not improve
 - 3s - loss: 0.2836 - acc: 0.8700 - val_loss: 0.2850 - val_acc: 0.8710
Epoch 112/150
Epoch 00112: val_loss did not improve
 - 3s - loss: 0.2554 - acc: 0.8750 - val_loss: 0.2827 - val_acc: 0.8654
Epoch 113/150
Epoch 00113: val_loss did not improve
 - 3s - loss: 0.2730 - acc: 0.8876 - val_loss: 0.3722 - val_acc: 0.8467
Epoch 11

Epoch 20/150
Epoch 00020: val_loss did not improve
 - 3s - loss: 0.3541 - acc: 0.8302 - val_loss: 0.4812 - val_acc: 0.7715
Epoch 21/150
Epoch 00021: val_loss did not improve
 - 3s - loss: 0.3465 - acc: 0.8455 - val_loss: 0.3916 - val_acc: 0.7959
Epoch 22/150
Epoch 00022: val_loss improved from 0.35598 to 0.34893, saving model to best_m.h5
 - 3s - loss: 0.3124 - acc: 0.8561 - val_loss: 0.3489 - val_acc: 0.8240
Epoch 23/150
Epoch 00023: val_loss did not improve
 - 3s - loss: 0.3554 - acc: 0.8325 - val_loss: 0.5693 - val_acc: 0.7041
Epoch 24/150
Epoch 00024: val_loss did not improve
 - 3s - loss: 0.3598 - acc: 0.8266 - val_loss: 0.3714 - val_acc: 0.8390
Epoch 25/150
Epoch 00025: val_loss did not improve
 - 3s - loss: 0.3470 - acc: 0.8343 - val_loss: 0.3847 - val_acc: 0.8090
Epoch 26/150
Epoch 00026: val_loss improved from 0.34893 to 0.33156, saving model to best_m.h5
 - 3s - loss: 0.3153 - acc: 0.8705 - val_loss: 0.3316 - val_acc: 0.8670
Epoch 27/150
Epoch 00027: val_loss did not improve


Epoch 85/150
Epoch 00085: val_loss did not improve
 - 3s - loss: 0.2628 - acc: 0.8796 - val_loss: 0.3562 - val_acc: 0.8221
Epoch 86/150
Epoch 00086: val_loss did not improve
 - 3s - loss: 0.2613 - acc: 0.8919 - val_loss: 0.3707 - val_acc: 0.8277
Epoch 87/150
Epoch 00087: val_loss did not improve
 - 3s - loss: 0.2987 - acc: 0.8598 - val_loss: 0.3491 - val_acc: 0.8240
Epoch 88/150
Epoch 00088: val_loss did not improve
 - 3s - loss: 0.2913 - acc: 0.8731 - val_loss: 0.3375 - val_acc: 0.8352
Epoch 89/150
Epoch 00089: val_loss did not improve
 - 3s - loss: 0.2604 - acc: 0.8785 - val_loss: 0.3204 - val_acc: 0.8614
Epoch 90/150
Epoch 00090: val_loss did not improve
 - 3s - loss: 0.2510 - acc: 0.8833 - val_loss: 0.3361 - val_acc: 0.8427
Epoch 91/150
Epoch 00091: val_loss did not improve
 - 3s - loss: 0.2546 - acc: 0.8797 - val_loss: 0.3228 - val_acc: 0.8464
Epoch 92/150
Epoch 00092: val_loss did not improve
 - 3s - loss: 0.2675 - acc: 0.8796 - val_loss: 0.3149 - val_acc: 0.8427
Epoch 93/150
Epo

In [18]:
with open('../features/cnn_5_aug_denoise_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
# 2364

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_5_aug_denoise_preprocess.csv', index=False)

0.260312679191
         id  is_iceberg
0  5941774d    0.325724
1  4023181e    0.253438
2  b20200e4    0.885919
3  e7f018bb    0.986684
4  4371c8c3    0.285046
